In [9]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
import math
from functools import partial
from open3d.t.geometry import TriangleMesh
import util

In [10]:
dataname = "/home/chris/Code/PointClouds/data/ply/CircularVentilationGrateExtraCleanedFull.ply"
pcd = o3d.io.read_point_cloud(dataname)
pcd = util.preProcessCloud(pcd)

In [11]:
def colorize_octree(octree, depth=8):
    colors = np.random.rand(2**depth, 3)  # Generate random colors for nodes

    def assign_color(node, node_info):
        if node_info.depth == depth:
            node.color = colors[node_info.child_index]  # Assign a color to each leaf node

    octree.traverse(assign_color)

# Define the octree depth
octree_depth = 8  # Higher depth = more subdivisions

# Create an octree and convert the point cloud into it
octree = o3d.geometry.Octree(max_depth=octree_depth)
octree.convert_from_point_cloud(pcd, size_expand=0.01)  # Expand slightly to ensure full coverage

# colorize_octree(octree, depth=octree_depth)
o3d.visualization.draw_geometries([octree])


In [12]:
def calculate_eigen_norm_and_plane_direction(neighbor_coordinates):
    if len(neighbor_coordinates) < 3:
        return np.array([0, 0, 0]), np.array([0, 0, 0]), np.array([0, 0, 0])
    
    mu = np.mean(neighbor_coordinates, axis=0)
    norm = neighbor_coordinates - mu
    cov = np.cov(norm.T)
    eig_val, eig_vec = np.linalg.eig(cov)
    sorted_idx = np.argsort(eig_val)[::-1]
    eig_val = eig_val[sorted_idx]
    eig_vec = eig_vec[:, sorted_idx]
    eig_val_norm = eig_val.copy()

    for z in range(len(eig_val)):
        eig_val_norm[z] = np.exp(eig_val[z])/np.sum(np.exp(eig_val))

    plane_direction = np.cross(eig_vec[:, 0], eig_vec[:, 1])

    return mu, eig_val_norm, plane_direction

In [13]:
import colorsys  # For HSV to RGB conversion

def plane_direction_to_color(plane_direction):
    """ Convert a plane normal (x, y, z) into an RGB color using a hue sphere mapping. """
    # Normalize direction vector
    plane_direction = plane_direction / np.linalg.norm(plane_direction)

    # Compute spherical coordinates
    theta = np.arctan2(plane_direction[1], plane_direction[0])  # Azimuth
    phi = np.arccos(plane_direction[2])  # Inclination

    # Normalize theta to [0,1] for hue (H)
    hue = (theta + np.pi) / (2 * np.pi)  # Map [-π, π] to [0,1]

    # Use fixed Saturation (S) and Value (V)
    saturation = 1.0
    value = 1.0

    # Convert HSV to RGB
    r, g, b = colorsys.hsv_to_rgb(hue, saturation, value)
    return [r, g, b]

def apply_pca_to_tree(pcd, octree, depth=8, vector_scale=0.4, min_points=5):
    lines = []
    colors = []
    all_points = np.asarray(pcd.points)
    def apply_pca(node, node_info):
        if node_info.depth == depth and isinstance(node, o3d.geometry.OctreeLeafNode):
            #print('Hi')
            if hasattr(node, "indices"):
                leaf_points = all_points[node.indices]

                if len(leaf_points) < min_points:
                    return
                
                mu, _, plane_direction = calculate_eigen_norm_and_plane_direction(leaf_points)
                # Compute start and end points of the line
                start_point = mu - (vector_scale / 2) * plane_direction
                end_point = mu + (vector_scale / 2) * plane_direction

                # Store the line
                lines.append([start_point, end_point])
                color = plane_direction_to_color(plane_direction)
                colors.append(color)
                colors.append(color)

    octree.traverse(apply_pca)
    return lines, colors


In [14]:
lines, colors = apply_pca_to_tree(pcd, octree, depth=octree_depth, vector_scale=0.4, min_points=5)

In [ ]:
import time

def visualize_lines(lines, colors, batch_size=10000):
    vis = o3d.visualization.Visualizer()
    vis.create_window()

    # Convert to Open3D format
    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(np.concatenate(lines, axis=0))

    # Fix line indexing
    line_indices = [[i, i + 1] for i in range(len(lines))]
    line_set.lines = o3d.utility.Vector2iVector(line_indices)

    # Ensure colors match line count
    line_set.colors = o3d.utility.Vector3dVector(colors)

    # Add in batches to prevent skipping
    for i in range(0, len(line_indices), batch_size):
        print(f"Rendering batch {i // batch_size + 1}")
        batch_lines = o3d.geometry.LineSet()
        batch_lines.points = line_set.points
        batch_lines.lines = o3d.utility.Vector2iVector(line_indices[i:i + batch_size])
        batch_lines.colors = o3d.utility.Vector3dVector(colors[i:i + batch_size])  # Fix here
        vis.add_geometry(batch_lines)
        vis.poll_events()
        vis.update_renderer()

    vis.run()
    vis.destroy_window()

# Run the visualization
visualize_lines(lines, colors)

Rendering batch 1
Rendering batch 2
Rendering batch 3
Rendering batch 4
Rendering batch 5
Rendering batch 6
